In [ ]:
'''Finite state machine idea

Wants to get from locationA to locationB, but if objects are in the way it must avoid them

States:
    -Driving from A to B (default)
    -Braking before an object (triggered by distance reading)
    -Orbiting an object (triggered by object detection)
    
Functions:
    drive_to_dest(location1,location2)
        -arguments: current coordinate location, and destination coordinate location.
        -actions: figure out what it's heading and driving distance is
        -returns: (heading, distance)
        
    scan_object()
        -arguments: none
        -actions: stop GoPiGo and scan the object in front
        -returns: (object_position, object_width)
       
    orbit_object(object_width)
        -arguments: object_width
        -actions: turn 90 deg right, orbit around that object using orbit function,
        turn 90 deg right again
        -returns: '''
        

In [4]:
#Init

##IMPORTS##
from easygopigo3 import EasyGoPiGo3
from EDL_Jupyter_resources import HiddenPrints
from IPython.display import clear_output
import time
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
hiddenprints=HiddenPrints()

# Initialize easy gpg
easyGPG = EasyGoPiGo3()
# Make sure that all sensors and output devices are uncofigured to start
easyGPG.reset_all()
easyGPG.reset_speed()

###SENSORS###
my_distance_sensor=easyGPG.init_distance_sensor()
my_servo = easyGPG.init_servo("SERVO1")

def normalize_min_max(raw_data):
    
    normalized_data=np.array([])
    
    max_reading=np.max(raw_data)
    min_reading=np.min(raw_data) 

    if max_reading==min_reading:
        return []
    
    for raw_reading in raw_data:
        normalized_reading = (raw_reading - min_reading)/(max_reading-min_reading) 
        normalized_data=np.append(normalized_data,normalized_reading)
    
    return normalized_data


print('Init.')

Init.


In [ ]:
##VARIABLES##
object_position=[300,0]
object_width=0
heading=0
step=-1
buffer=15
sensor_offset=15
MAP=[
    [0,0],
    [100,0]
]
objects=[]

def drive(heading, location1, location2):
    #Use the distance formula to find the distance to drive
    dx=location2[0]-location1[0]
    dy=location2[1]-location1[1]
    dist = np.sqrt((dx)**2 + (dy)**2)
    
    #Use the arctan function to find the abs angle to turn
    if dx>0:
        abs_angle = (np.arctan((dy/dx)))*180/(np.pi)
    elif dx<0:
        abs_angle = 180+(np.arctan((dy/dx)))*180/(np.pi)
    else:
        if dy>0:
            abs_angle=90
        else:
            abs_angle=-90
    
    angle=abs_angle-heading #angle to turn is absolute angle - heading angle
    
    easyGPG.turn_degrees(-angle) #turn 
    heading=abs_angle #set heading to where we are facing now
    
    easyGPG.set_speed(300) #set speed
    easyGPG.reset_encoders() #reset encoders so we know how far we have driven
    easyGPG.drive_cm(dist,blocking=False) #begin driving, but allow us to be interrupted
    
    return(heading)

def scan_object(graph=False):

    ###SCAN PARAMETERS###
    scan_center=100
    scan_step=5
    scan_width=120
    zero_position=scan_center-(scan_width//2)

    ###ARRAY INIT.###
    distances=np.array([])
    angles=np.array([])

    ###SCAN###
    for i in range(scan_width//scan_step):
        #scan from right to left
        angle=zero_position+i*scan_step
        my_servo.rotate_servo(angle)
        distances=np.append(distances,my_distance_sensor.read())
        angles=np.append(angles,angle)
        time.sleep(0.01)

    ###OBJECT WIDTH CALCULATION###
    normalized_distances=normalize_min_max(distances) #normalize distance data
    if len(normalized_distances)==0:
        return([300,0],0)
    object_angles=angles[np.where(normalized_distances<0.1)] #get the angles where it 'sees the object'
    object_angular_width=np.max(object_angles)-np.min(object_angles) #calculate the width of that range of angles
    min_dist=distances[np.where(angles==object_angles[len(object_angles)//2])] #find the distance to the 'center' of the object
    object_width=2*min_dist*np.tan((object_angular_width*np.pi/180)/2) #do trig to figure out the object width
    
    ###OBJECT LOCATION CALCULATION###
    min_angle=np.average(object_angles) #min angle is center of object_angles
    object_heading=min_angle-scan_center
    dx = min_dist*np.cos((object_heading*np.pi/180)) #dx found using trig
    dy = min_dist*np.sin((object_heading*np.pi/180)) #dy found using trig (note left of bot is positive)
 
    if graph:
        plt.figure(0)
        plt.plot(angles,distances)
        plt.plot(min_angle,min_dist,marker='o', markersize=5, color="red")
        plt.show()
    
    return([dx,dy],object_width)

def avoid(heading, location1, object_location, object_width):
    
    object_dist = object_location[0] #see how close our object is
    dI = easyGPG.read_encoders_average() #get distance driven before object
    easyGPG.reset_encoders() #reset the encoders before our orbit
    
    orbit_radius = object_dist+object_width/2+sensor_offset #calculate how large of an orbit to make
    
    if object_location[1]>=0: #if object is to our left, go right or if it is center
        easyGPG.turn_degrees(90)
        easyGPG.orbit(-180,orbit_radius) #orbit the object
        easyGPG.turn_degrees(90)
    elif object_location[1]<0:
        easyGPG.turn_degrees(-90)
        easyGPG.orbit(180,orbit_radius) #orbit the object
        easyGPG.turn_degrees(-90)
    
    dO = (easyGPG.read_encoders_average()*2)/np.pi #get distance driven "through" object
    easyGPG.reset_encoders() #reset encoders

    dx = (dI+dO)*(np.cos(heading*np.pi/180)) #calculate the change in x and y 
    dy = (dI+dO)*(np.sin(heading*np.pi/180))

    location1 = [location1[0]+dx,location1[1]+dy] #update location
    objects.append([location1[0]-dO*np.cos(heading*np.pi/180),location1[1]-dO*np.sin(heading*np.pi/180)])
    
    return(location1)

###STATES###
driving=True
avoiding=False

###MAIN LOOP###
while True:
    if not driving and not avoiding:
        avoiding=False
        print('Driving.')
        driving=True
        heading=drive(heading,location1,location2)
        (object_position,object_width)=scan_object()
    elif driving and object_position[0]<buffer:
        print('Found object, scanning.')
        easyGPG.stop()
        driving=False
        (object_position,object_width)=scan_object()
        print('Going around object')
        avoiding=True
        location1=avoid(heading,location1,object_position, object_width)
        avoiding=False
    elif driving and not avoiding:
        first=easyGPG.read_encoders_average()
        time.sleep(0.1)
        second=easyGPG.read_encoders_average()
        if first==second:
            print('Finished move.')
            step=step+1
            driving = False
            easyGPG.stop()
            location1=MAP[step]
            try:
                location2=MAP[step+1]
            except IndexError:
                break

easyGPG.stop()
print('Done.')

plt.figure()
for i in range(len(MAP)):
    plt.plot(MAP[i][0],MAP[i][1],marker='o', markersize=5, color="blue")
for i in range(len(objects)):
    plt.plot(objects[i][0],objects[i][1],marker='o', markersize=10, color="red")
plt.show()

In [2]:
easyGPG.forward()
time.sleep(1)
easyGPG.stop()